# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [283]:
import pandas as pd
import numpy as np
import math
import json
import sagemaker
import os
import seaborn as sn
import matplotlib.pyplot as plt

%matplotlib inline

data_dir = 'data/'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [200]:
# read in the json files
portfolio = pd.read_json(os.path.join(data_dir, 'portfolio.json'), orient='records', lines=True)
profile = pd.read_json(os.path.join(data_dir, 'profile.json'), orient='records', lines=True)
transcript = pd.read_json(os.path.join(data_dir, 'transcript.json'), orient='records', lines=True)

In [201]:
portfolio['class'] = portfolio.index

In [202]:
portfolio.shape[0]

10

In [203]:
portfolio

reward                      channels  difficulty  duration     offer_type  \
0      10       [email, mobile, social]          10         7           bogo   
1      10  [web, email, mobile, social]          10         5           bogo   
2       0          [web, email, mobile]           0         4  informational   
3       5          [web, email, mobile]           5         7           bogo   
4       5                  [web, email]          20        10       discount   
5       3  [web, email, mobile, social]           7         7       discount   
6       2  [web, email, mobile, social]          10        10       discount   
7       0       [email, mobile, social]           0         3  informational   
8       5  [web, email, mobile, social]           5         5           bogo   
9       2          [web, email, mobile]          10         7       discount   

                                 id  class  
0  ae264e3637204a6fb9bb56bc8210ddfd      0  
1  4d5c57ea9a6940dd891ad53e9dbe8da0      1  
2  3f207df678b143eea3cee63160fa8bed      2  
3  9b98b8c7a33c4b65b9aebfe6a799e6d9      3  
4  0b1e1539f2cc45b7b9fa7c272da2e1d7      4  
5  2298d6c36e964ae4a3e7e9706d1fb8c2      5  
6  fafdcd668e3743c1bb461111dcafc2a4      6  
7  5a8bc65990b245e5a138643cd4eb9837      7  
8  f19421c1d4aa40978ebb69ca19b0e20d      8  
9  2906b810c7d4411798c6938adc9daaa5      9

In [204]:
profile.head()

gender  age                                id  became_member_on    income
0   None  118  68be06ca386d4c31939f3a4f0e3dd783          20170212       NaN
1      F   55  0610b486422d4921ae7d2bf64640c50b          20170715  112000.0
2   None  118  38fe809add3b4fcf9315a9694bb96ff5          20180712       NaN
3      F   75  78afa995795e4d85b5d9ceeca43f5fef          20170509  100000.0
4   None  118  a03223e636434f42ac4c3df47e8bac43          20170804       NaN

In [205]:
transcript['event'].unique()

array(['offer received', 'offer viewed', 'transaction', 'offer completed'],
      dtype=object)

In [206]:
print("offer recived: {}".format(transcript[transcript['event'] == 'offer received'].shape[0]))
print("offer viewed: {}".format(transcript[transcript['event'] == 'offer viewed'].shape[0]))
print("transaction: {}".format(transcript[transcript['event'] == 'transaction'].shape[0]))
print("offer completed: {}".format(transcript[transcript['event'] == 'offer completed'].shape[0]))

offer recived: 76277
offer viewed: 57725
transaction: 138953
offer completed: 33579


In [207]:
transcript.head()

person           event  \
0  78afa995795e4d85b5d9ceeca43f5fef  offer received   
1  a03223e636434f42ac4c3df47e8bac43  offer received   
2  e2127556f4f64592b11af22de27a7932  offer received   
3  8ec6ce2a7e7949b1bf142def7d0e0586  offer received   
4  68617ca6246f4fbc85e91a2a49552598  offer received   

                                              value  time  
0  {'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}     0  
1  {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}     0  
2  {'offer id': '2906b810c7d4411798c6938adc9daaa5'}     0  
3  {'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}     0  
4  {'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}     0

In [208]:
transcript[transcript['event'] == 'transaction'].head()

person        event  \
12654  02c083884c7d45b39cc68e1314fec56c  transaction   
12657  9fa9ae8f57894cc9a3b8a9bbe0fc1b2f  transaction   
12659  54890f68699049c2a04d415abc25e717  transaction   
12670  b2f1cd155b864803ad8334cdf13c4bd2  transaction   
12671  fe97aa22dd3e48c8b143116a8403dd52  transaction   

                                value  time  
12654  {'amount': 0.8300000000000001}     0  
12657               {'amount': 34.56}     0  
12659               {'amount': 13.23}     0  
12670               {'amount': 19.51}     0  
12671               {'amount': 18.97}     0

In [209]:
transcript[transcript['event'] == 'transaction'].head().iloc[0,2]

{'amount': 0.8300000000000001}

In [210]:
transcript[transcript['event'] == 'offer received'].head()

person           event  \
0  78afa995795e4d85b5d9ceeca43f5fef  offer received   
1  a03223e636434f42ac4c3df47e8bac43  offer received   
2  e2127556f4f64592b11af22de27a7932  offer received   
3  8ec6ce2a7e7949b1bf142def7d0e0586  offer received   
4  68617ca6246f4fbc85e91a2a49552598  offer received   

                                              value  time  
0  {'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}     0  
1  {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}     0  
2  {'offer id': '2906b810c7d4411798c6938adc9daaa5'}     0  
3  {'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}     0  
4  {'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}     0

In [211]:
transcript[transcript['event'] == 'offer received'].head().iloc[0,2]

{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}

In [212]:
transcript[transcript['event'] == 'offer viewed'].head()

person         event  \
12650  389bc3fa690240e798340f5a15918d5c  offer viewed   
12651  d1ede868e29245ea91818a903fec04c6  offer viewed   
12652  102e9454054946fda62242d2e176fdce  offer viewed   
12653  02c083884c7d45b39cc68e1314fec56c  offer viewed   
12655  be8a5d1981a2458d90b255ddc7e0d174  offer viewed   

                                                  value  time  
12650  {'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'}     0  
12651  {'offer id': '5a8bc65990b245e5a138643cd4eb9837'}     0  
12652  {'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}     0  
12653  {'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'}     0  
12655  {'offer id': '5a8bc65990b245e5a138643cd4eb9837'}     0

In [213]:
transcript[transcript['event'] == 'offer viewed'].head().iloc[0,2]['offer id']

'f19421c1d4aa40978ebb69ca19b0e20d'

In [214]:
transcript[transcript['event'] == 'offer completed'].head()

person            event  \
12658  9fa9ae8f57894cc9a3b8a9bbe0fc1b2f  offer completed   
12672  fe97aa22dd3e48c8b143116a8403dd52  offer completed   
12679  629fc02d56414d91bca360decdfa9288  offer completed   
12692  676506bad68e4161b9bbaffeb039626b  offer completed   
12697  8f7dd3b2afe14c078eb4f6e6fe4ba97d  offer completed   

                                                   value  time  
12658  {'offer_id': '2906b810c7d4411798c6938adc9daaa5...     0  
12672  {'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...     0  
12679  {'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...     0  
12692  {'offer_id': 'ae264e3637204a6fb9bb56bc8210ddfd...     0  
12697  {'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0...     0

In [215]:
transcript[transcript['event'] == 'offer completed'].head().iloc[0,2]

{'offer_id': '2906b810c7d4411798c6938adc9daaa5', 'reward': 2}

In [216]:
completed_transcript = transcript[transcript['event'] == 'offer completed']
viewed_transcript = transcript[transcript['event'] == 'offer viewed']
recieved_transcript = transcript[transcript['event'] == 'offer received']
transaction_transcript = transcript[transcript['event'] == 'transaction']

completed_transcript = completed_transcript.reset_index(drop=True)
viewed_transcript = viewed_transcript.reset_index(drop=True)
recieved_transcript = recieved_transcript.reset_index(drop=True)
transaction_transcript = transaction_transcript.reset_index(drop=True)

viewed_transcript['value'] = [x['offer id'] for x in viewed_transcript['value']]
recieved_transcript['value'] = [x['offer id'] for x in recieved_transcript['value']]
completed_transcript['reward'] = [x['reward'] for x in completed_transcript['value']]
completed_transcript['value'] = [x['offer_id'] for x in completed_transcript['value']]
transaction_transcript['value'] = [x['amount'] for x in transaction_transcript['value']]

In [217]:
recieved_transcript.head()

person           event  \
0  78afa995795e4d85b5d9ceeca43f5fef  offer received   
1  a03223e636434f42ac4c3df47e8bac43  offer received   
2  e2127556f4f64592b11af22de27a7932  offer received   
3  8ec6ce2a7e7949b1bf142def7d0e0586  offer received   
4  68617ca6246f4fbc85e91a2a49552598  offer received   

                              value  time  
0  9b98b8c7a33c4b65b9aebfe6a799e6d9     0  
1  0b1e1539f2cc45b7b9fa7c272da2e1d7     0  
2  2906b810c7d4411798c6938adc9daaa5     0  
3  fafdcd668e3743c1bb461111dcafc2a4     0  
4  4d5c57ea9a6940dd891ad53e9dbe8da0     0

In [218]:
viewed_transcript.head()

person         event  \
0  389bc3fa690240e798340f5a15918d5c  offer viewed   
1  d1ede868e29245ea91818a903fec04c6  offer viewed   
2  102e9454054946fda62242d2e176fdce  offer viewed   
3  02c083884c7d45b39cc68e1314fec56c  offer viewed   
4  be8a5d1981a2458d90b255ddc7e0d174  offer viewed   

                              value  time  
0  f19421c1d4aa40978ebb69ca19b0e20d     0  
1  5a8bc65990b245e5a138643cd4eb9837     0  
2  4d5c57ea9a6940dd891ad53e9dbe8da0     0  
3  ae264e3637204a6fb9bb56bc8210ddfd     0  
4  5a8bc65990b245e5a138643cd4eb9837     0

In [219]:
completed_transcript.head()

person            event  \
0  9fa9ae8f57894cc9a3b8a9bbe0fc1b2f  offer completed   
1  fe97aa22dd3e48c8b143116a8403dd52  offer completed   
2  629fc02d56414d91bca360decdfa9288  offer completed   
3  676506bad68e4161b9bbaffeb039626b  offer completed   
4  8f7dd3b2afe14c078eb4f6e6fe4ba97d  offer completed   

                              value  time  reward  
0  2906b810c7d4411798c6938adc9daaa5     0       2  
1  fafdcd668e3743c1bb461111dcafc2a4     0       2  
2  9b98b8c7a33c4b65b9aebfe6a799e6d9     0       5  
3  ae264e3637204a6fb9bb56bc8210ddfd     0      10  
4  4d5c57ea9a6940dd891ad53e9dbe8da0     0      10

In [220]:
transaction_transcript.head()

person        event  value  time
0  02c083884c7d45b39cc68e1314fec56c  transaction   0.83     0
1  9fa9ae8f57894cc9a3b8a9bbe0fc1b2f  transaction  34.56     0
2  54890f68699049c2a04d415abc25e717  transaction  13.23     0
3  b2f1cd155b864803ad8334cdf13c4bd2  transaction  19.51     0
4  fe97aa22dd3e48c8b143116a8403dd52  transaction  18.97     0

In [221]:
recieved_reviewed_completed = pd.DataFrame(columns = recieved_transcript.columns)

recieved_reviewed = pd.merge(viewed_transcript, recieved_transcript, on=['person','value'], how='inner')
recieved_reviewed_completed = pd.merge(recieved_reviewed, completed_transcript, on=['person','value'], how='inner')

time_valid = (recieved_reviewed_completed['time_x'] <= recieved_reviewed_completed['time_y']) & (recieved_reviewed_completed['time_y'] <= recieved_reviewed_completed['time'])
recieved_reviewed_completed = recieved_reviewed_completed[time_valid]

recieved_reviewed_completed = recieved_reviewed_completed.drop(['event_x','time_x','event_y','time_y','event','time', 'reward'], axis=1)

In [222]:
recieved_reviewed_completed.head()

person                             value
0  389bc3fa690240e798340f5a15918d5c  f19421c1d4aa40978ebb69ca19b0e20d
1  389bc3fa690240e798340f5a15918d5c  f19421c1d4aa40978ebb69ca19b0e20d
3  389bc3fa690240e798340f5a15918d5c  f19421c1d4aa40978ebb69ca19b0e20d
7  389bc3fa690240e798340f5a15918d5c  f19421c1d4aa40978ebb69ca19b0e20d
8  102e9454054946fda62242d2e176fdce  4d5c57ea9a6940dd891ad53e9dbe8da0

In [223]:
recieved_reviewed_not_completed = pd.DataFrame(columns = recieved_transcript.columns)

recieved_reviewed = pd.merge(viewed_transcript, recieved_transcript, on=['person','value'], how='inner')
recieved_reviewed_not_completed = pd.merge(recieved_reviewed, completed_transcript, on=['person','value'], how='left')

recieved_reviewed_not_completed = recieved_reviewed_not_completed[recieved_reviewed_not_completed['event'].isnull()]

time_valid = (recieved_reviewed_not_completed['time_x'] <= recieved_reviewed_not_completed['time_y']) 
recieved_reviewed_not_completed = recieved_reviewed_not_completed[time_valid]

recieved_reviewed_not_completed = recieved_reviewed_not_completed.drop(['event_x','time_x','event_y','time_y','event','time', 'reward'], axis=1)

In [224]:
recieved_reviewed_not_completed.head()

person                             value
8   d1ede868e29245ea91818a903fec04c6  5a8bc65990b245e5a138643cd4eb9837
10  02c083884c7d45b39cc68e1314fec56c  ae264e3637204a6fb9bb56bc8210ddfd
11  be8a5d1981a2458d90b255ddc7e0d174  5a8bc65990b245e5a138643cd4eb9837
14  e528ceb341964128aaf58a59733ec2af  2298d6c36e964ae4a3e7e9706d1fb8c2
15  e528ceb341964128aaf58a59733ec2af  2298d6c36e964ae4a3e7e9706d1fb8c2

In [225]:
recieved_not_reviewed = pd.DataFrame(columns = recieved_transcript.columns)

recieved_not_reviewed = pd.merge(recieved_transcript, viewed_transcript, on=['person','value'], how='left')

recieved_not_reviewed = recieved_not_reviewed[recieved_not_reviewed['event_y'].isnull()]

recieved_not_reviewed = recieved_not_reviewed.drop(['event_x','time_x','event_y','time_y'], axis=1)

In [226]:
recieved_not_reviewed.head()

person                             value
11  2eeac8d8feae4a8cad5a6af0499a211d  3f207df678b143eea3cee63160fa8bed
26  65aba5c617294649aeb624da249e1ee5  2906b810c7d4411798c6938adc9daaa5
29  868317b9be554cb18e50bc68484749a2  2906b810c7d4411798c6938adc9daaa5
32  4beeb3ed64dd4898b0edf2f6b67426d3  2906b810c7d4411798c6938adc9daaa5
34  25c906289d154b66bf579693f89481c9  2906b810c7d4411798c6938adc9daaa5

In [227]:
completed_not_viewed = pd.DataFrame(columns = recieved_transcript.columns)

completed_not_viewed = pd.merge(completed_transcript, viewed_transcript, on=['person','value'], how='left')

completed_not_viewed = completed_not_viewed[completed_not_viewed['event_y'].isnull()]

completed_not_viewed = completed_not_viewed.drop(['event_x','time_x','event_y','time_y','reward'], axis=1)

In [228]:
completed_not_viewed.head()

person                             value
12  b860d355ef6e4c66b5d5a837c56ef32d  9b98b8c7a33c4b65b9aebfe6a799e6d9
14  24115a61df25473e84a8a03f3c98de1a  2906b810c7d4411798c6938adc9daaa5
19  2481f1fcfbcb4b288e5a03af02d95373  2906b810c7d4411798c6938adc9daaa5
28  16dc4306258b4fef90ffe99f53ed7952  9b98b8c7a33c4b65b9aebfe6a799e6d9
29  2c9a9137efff4820a47d56dbf0fa555f  f19421c1d4aa40978ebb69ca19b0e20d

In [229]:
# good_offers_with_labels =
bad_offers = recieved_reviewed_not_completed.append(completed_not_viewed.append(recieved_not_reviewed))
good_offers = recieved_reviewed_completed

In [230]:
good_offers.head()

person                             value
0  389bc3fa690240e798340f5a15918d5c  f19421c1d4aa40978ebb69ca19b0e20d
1  389bc3fa690240e798340f5a15918d5c  f19421c1d4aa40978ebb69ca19b0e20d
3  389bc3fa690240e798340f5a15918d5c  f19421c1d4aa40978ebb69ca19b0e20d
7  389bc3fa690240e798340f5a15918d5c  f19421c1d4aa40978ebb69ca19b0e20d
8  102e9454054946fda62242d2e176fdce  4d5c57ea9a6940dd891ad53e9dbe8da0

In [231]:
bad_offers.head()

person                             value
8   d1ede868e29245ea91818a903fec04c6  5a8bc65990b245e5a138643cd4eb9837
10  02c083884c7d45b39cc68e1314fec56c  ae264e3637204a6fb9bb56bc8210ddfd
11  be8a5d1981a2458d90b255ddc7e0d174  5a8bc65990b245e5a138643cd4eb9837
14  e528ceb341964128aaf58a59733ec2af  2298d6c36e964ae4a3e7e9706d1fb8c2
15  e528ceb341964128aaf58a59733ec2af  2298d6c36e964ae4a3e7e9706d1fb8c2

In [232]:
def fetch_person_offer_data(data,offers,profile,set_class=False):
    data_offers = pd.merge(data, profile, left_on = 'person', right_on = 'id', how='inner')
    data_offers_profile = pd.merge(data_offers, offers, left_on = 'value', right_on = 'id', how='inner')
    if set_class == False:
        data_offers_profile['class'] = 0 
    return data_offers_profile.drop(['person','value','id_x','id_y','channels','difficulty','duration','offer_type','reward'], axis=1)

In [233]:
bad_offers = fetch_person_offer_data(bad_offers,portfolio,profile,set_class=True)
good_offers = fetch_person_offer_data(good_offers,portfolio,profile,set_class=True)

In [234]:
bad_offers.head()

gender  age  became_member_on   income  class
0      O   53          20170916  52000.0      7
1      M   39          20140527  51000.0      7
2      M   40          20180227  53000.0      7
3   None  118          20170823      NaN      7
4      F   51          20170813  56000.0      7

In [235]:
good_offers.head()

gender  age  became_member_on   income  class
0      M   65          20180209  53000.0      8
1      M   65          20180209  53000.0      8
2      M   65          20180209  53000.0      8
3      M   65          20180209  53000.0      8
4      F   39          20170919  60000.0      8

In [236]:
data = good_offers.append(bad_offers)
data = data.sample(frac=1).reset_index(drop=True)
data.dropna(subset = data.columns.values, inplace=True)

In [237]:
data.head()

gender  age  became_member_on   income  class
0      F   66          20160124  73000.0      0
1      F   59          20160106  74000.0      2
3      M   25          20171008  34000.0      2
4      F   58          20170409  74000.0      9
5      M   47          20171202  35000.0      3

In [238]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40197 entries, 0 to 45284
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            40197 non-null  object 
 1   age               40197 non-null  int64  
 2   became_member_on  40197 non-null  int64  
 3   income            40197 non-null  float64
 4   class             40197 non-null  int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 1.8+ MB


In [284]:
corrMatrix = data.corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()

In [239]:
print(["class {} : {}".format(i,data[data['class'] == i].shape[0]) for i in range(10)]) 

['class 0 : 3321', 'class 1 : 3121', 'class 2 : 4039', 'class 3 : 5407', 'class 4 : 6495', 'class 5 : 3334', 'class 6 : 3298', 'class 7 : 2811', 'class 8 : 3157', 'class 9 : 5214']


In [240]:
genders = { key : index for index,key in enumerate(list(data['gender'].unique())) } 

data['gender'] = [ genders[x] for x in data['gender'] ]

In [241]:
data.head()

gender  age  became_member_on   income  class
0       0   66          20160124  73000.0      0
1       0   59          20160106  74000.0      2
3       1   25          20171008  34000.0      2
4       0   58          20170409  74000.0      9
5       1   47          20171202  35000.0      3

In [242]:
samples = data.loc[:,['age','became_member_on','gender','income']]
labels = data.loc[:,['class']]

In [243]:
samples.head()

age  became_member_on  gender   income
0   66          20160124       0  73000.0
1   59          20160106       0  74000.0
3   25          20171008       1  34000.0
4   58          20170409       0  74000.0
5   47          20171202       1  35000.0

In [244]:
labels.head()

class
0      0
1      2
3      2
4      9
5      3

In [81]:
from sklearn import preprocessing

def one_hot_encoding(labels):

    enc = preprocessing.OneHotEncoder()

    enc.fit(labels)

    # 3. Transform
    onehotlabels = enc.transform(labels).toarray()
    
    return onehotlabels

In [88]:
one_hot_encoded = one_hot_encoding(labels)

In [89]:
one_hot_encoded

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [257]:
from sklearn.preprocessing import MinMaxScaler

samples_array = samples.values

scaler = MinMaxScaler()
scaler.fit(samples_array)

samples_array = scaler.fit_transform(samples_array)

samples_columns = samples.columns
samples = pd.DataFrame(samples_array)
samples.columns = samples_columns

In [258]:
samples.describe()

age  became_member_on       gender        income
count  40197.000000      40197.000000  40197.00000  40197.000000
mean       0.440766          0.720711      0.29445      0.398514
std        0.210740          0.232305      0.25924      0.243445
min        0.000000          0.000000      0.00000      0.000000
25%        0.289157          0.595776      0.00000      0.211111
50%        0.457831          0.799868      0.50000      0.377778
75%        0.590361          0.809909      0.50000      0.555556
max        1.000000          1.000000      1.00000      1.000000

In [259]:
samples.head()

age  became_member_on  gender    income
0  0.578313          0.587935     0.0  0.477778
1  0.493976          0.587575     0.0  0.488889
2  0.084337          0.805628     0.5  0.044444
3  0.481928          0.793648     0.0  0.488889
4  0.349398          0.809509     0.5  0.055556

In [260]:
from sklearn.model_selection import train_test_split

train_X, test_X , train_Y, test_Y = train_test_split(samples.values, labels.values, test_size=0.33, random_state=1)

In [261]:
print("size of train_X , {}".format(len(train_X)))
print("size of train_Y , {}".format(len(train_Y)))
print("size of test_X , {}".format(len(test_X)))
print("size of test_Y , {}".format(len(test_Y)))

size of train_X , 26931
size of train_Y , 26931
size of test_X , 13266
size of test_Y , 13266


In [262]:
train_Y

array([[4],
       [7],
       [8],
       ...,
       [6],
       [3],
       [9]])

In [263]:
pd.DataFrame(test_X).to_csv(os.path.join(data_dir, 'test.csv'), header=False, index=False)

pd.concat([pd.DataFrame(train_Y), pd.DataFrame(train_X)], axis=1).to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

In [264]:
pd.concat([pd.DataFrame(train_Y), pd.DataFrame(train_X)], axis=1)

0         0         1    2         3
0      4  0.626506  0.409805  0.5  0.222222
1      7  0.650602  0.806008  0.5  0.411111
2      8  0.771084  0.993700  0.5  0.388889
3      7  0.084337  0.803928  0.0  0.366667
4      5  0.493976  1.000000  0.5  0.044444
...   ..       ...       ...  ...       ...
26926  9  0.602410  0.609737  0.5  0.666667
26927  4  0.843373  0.799948  0.0  0.655556
26928  6  0.759036  0.795948  0.5  0.422222
26929  3  0.397590  0.789687  0.5  0.722222
26930  9  0.361446  0.593496  0.0  0.088889

[26931 rows x 5 columns]

In [265]:
session = sagemaker.Session() # Store the current SageMaker session

# S3 prefix (which folder will we use)
prefix = 'starbucks-recommender-model'

test_location = session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)
train_location = session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)

In [266]:
# To save a bit of memory we can set text_X, train_X, val_X, train_y and val_y to None.

train_X = train_Y = None

In [267]:
from sagemaker import get_execution_role

# Our current execution role is require when creating the model as the training
# and inference code will need to access the model artifacts.
role = get_execution_role()

In [268]:
# We need to retrieve the location of the container which is provided by Amazon for using XGBoost.
# As a matter of convenience, the training and inference code both use the same container.
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(session.boto_region_name, 'xgboost')

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
There is a more up to date SageMaker XGBoost image. To use the newer image, please set 'repo_version'='1.0-1'. For example:
	get_image_uri(region, 'xgboost', '1.0-1').


In [269]:
xgb = sagemaker.estimator.Estimator(container, # The location of the container we wish to use
                                    role,                                    # What is our current IAM Role
                                    train_instance_count=1,                  # How many compute instances
                                    train_instance_type='ml.m4.xlarge',      # What kind of compute instances
                                    output_path='s3://{}/{}/output'.format(session.default_bucket(), prefix),
                                    sagemaker_session=session)

xgb.set_hyperparameters(min_child_weight=5,
                        silent=0,
                        objective='multi:softmax',
                        num_class=10,
                        num_round=25)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [270]:
s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='csv')

xgb.fit({'train': s3_input_train})

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-08-22 22:06:02 Starting - Starting the training job...
2020-08-22 22:06:04 Starting - Launching requested ML instances......
2020-08-22 22:07:08 Starting - Preparing the instances for training...
2020-08-22 22:07:56 Downloading - Downloading input data...
2020-08-22 22:08:31 Training - Training image download completed. Training in progress..Arguments: train
[2020-08-22:22:08:31:INFO] Running standalone xgboost training.
[2020-08-22:22:08:31:INFO] Path /opt/ml/input/data/validation does not exist!
[2020-08-22:22:08:31:INFO] File size need to be processed in the node: 1.62mb. Available memory size in the node: 8494.87mb
[2020-08-22:22:08:31:INFO] Determined delimiter of CSV input is ','
[22:08:31] S3DistributionType set as FullyReplicated
[22:08:32] 26931x4 matrix with 107724 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[22:08:32] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 84 extra nodes, 8 pruned nodes, max_depth=6
[22:08:32] src

In [271]:
xgb_transformer = xgb.transformer(instance_count = 1, instance_type = 'ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


In [272]:
xgb_transformer.transform(test_location, content_type='text/csv', split_type='Line')

In [273]:
xgb_transformer.wait()

...............................Arguments: serve
[2020-08-22 22:15:21 +0000] [1] [INFO] Starting gunicorn 19.7.1
[2020-08-22 22:15:21 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2020-08-22 22:15:21 +0000] [1] [INFO] Using worker: gevent
[2020-08-22 22:15:21 +0000] [37] [INFO] Booting worker with pid: 37
[2020-08-22 22:15:21 +0000] [38] [INFO] Booting worker with pid: 38
[2020-08-22 22:15:21 +0000] [39] [INFO] Booting worker with pid: 39
[2020-08-22 22:15:21 +0000] [40] [INFO] Booting worker with pid: 40
[2020-08-22:22:15:21:INFO] Model loaded successfully for worker : 37
[2020-08-22:22:15:21:INFO] Model loaded successfully for worker : 38
[2020-08-22:22:15:21:INFO] Model loaded successfully for worker : 39
[2020-08-22:22:15:21:INFO] Model loaded successfully for worker : 40
[2020-08-22:22:15:22:INFO] Sniff delimiter as ','
[2020-08-22:22:15:22:INFO] Determined delimiter of CSV input is ','
2020-08-22T22:15:21.872:[sagemaker logs]: MaxConcurrentTransforms=4, MaxPayloadInMB=6

In [274]:
!aws s3 cp --recursive $xgb_transformer.output_path $data_dir

download: s3://sagemaker-us-east-2-613763915925/xgboost-2020-08-22-22-10-16-741/test.csv.out to data/test.csv.out


In [275]:
predictions = pd.read_csv(os.path.join(data_dir, 'test.csv.out'), header=None)

In [276]:
predictions.head()

0
0  9.0
1  3.0
2  3.0
3  3.0
4  4.0

In [277]:
test_Y

array([[0],
       [8],
       [4],
       ...,
       [6],
       [4],
       [9]])

In [278]:
from sklearn.metrics import precision_recall_fscore_support
from statistics import mean 

prec, rec, fbeta_test, support = precision_recall_fscore_support(test_Y, predictions.values, average = None)

print("recall : {}".format(mean(rec)))
print("f1 score : {}".format(mean(fbeta_test)))

recall : 0.14977841905146397
f1 score : 0.13836488103933156


In [281]:
summ

2577